In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import allantools

pd.set_option("display.precision", 15)      # Show up to 15 decimal places
baseName = 'fixedPos1'
dirName  = 'labData/'
rowLimit = 2000 # Rows to keep after joining ticc and timTp samples

# File format for .ticc.csv files logging PPS events
# Columns:
#  ppsHostClock: Host clock when PPS event timestamp arrvied (UTC)
#  ppsRefClock:  Reference clock when PPS event happened (zero-based count of elapsed seconds)

ticc = pd.read_csv(f"{dirName}/{baseName}.ticcA.csv")
ticc['fn'] = baseName

# Convert string to datetime
ticc["ppsHostClock"] = pd.to_datetime(ticc.ppsHostClock, utc=True)

# Assuming host clock sync is better than serialization latency of timestamp arriving,
# floor of host clock second will be the navigation epoch sencond.
# Will be used for later join with TIM-TP timestamps.
ticc["epochSec"] = ticc["ppsHostClock"].dt.floor("s")


In [ ]:
ticc.dtypes

In [ ]:
ticc

In [ ]:
ticc['rcFrac'] = ticc.ppsRefClock -ticc.ppsRefClock .astype('int')  # Get fractional part of refClock
ticc['hcFrac'] = (ticc.ppsHostClock.astype('int64')-1e9*(ticc.ppsHostClock.astype('int64')//1e9))/1e9  # Get fractional part of hostClock

In [ ]:
ticc['rcTi'] = ticc.ppsRefClock - ticc.ppsRefClock.shift(1) # Time interval between refClock samples on ref clock

In [ ]:
plt.figure(figsize=(18, 6))
plt.hist(ticc[(ticc.rcTi>0.999999999809074) & (ticc.rcTi<0.999999999850000)][:99900].rcTi, bins=400, color='blue', alpha=0.7)
plt.title('Histogram of rcTi')
plt.xlabel('rcTi')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)
plt.show()


In [ ]:
ticc.rcTi

In [ ]:
# np.savetxt('run1.snap1.ucTie.txt', ticc.rcFrac, fmt='%.12f')

In [ ]:
ticc["rcFracMa6"   ] = ticc["rcFrac"].rolling(window=6   ).mean()
ticc["rcFracMa60"  ] = ticc["rcFrac"].rolling(window=60  ).mean()
ticc["rcFracMa600" ] = ticc["rcFrac"].rolling(window=600 ).mean()
ticc["rcFracMa6000"] = ticc["rcFrac"].rolling(window=6000).mean()

ticc["rcFracMstd6"   ] = ticc["rcFrac"].rolling(window=6   ).std()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(ticc['ppsHostClock'], ticc['rcFracMstd6'], label='rcFracMstd6', marker='o')
plt.xlabel('ppsHostClock')
plt.ylabel('rcFracMstd6 Values')
plt.title('Plot of rcFracMstd6 over Time')
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.hist(ticc['rcFracMstd6'].dropna(), bins=30, color='blue', alpha=0.7)
plt.xlabel('rcFracMstd6 Values')
plt.ylabel('Frequency')
plt.title('Histogram of rcFracMstd6')
plt.grid()
plt.show()

In [ ]:
ticc

In [ ]:
plt.figure(figsize=(18, 6))
plt.hist(ticc['rcFrac'][:50].dropna(), bins=300, color='blue', alpha=0.7)
plt.xlabel('rcFrac Values')
plt.ylabel('Frequency')
plt.title('Histogram of rcFrac')
plt.grid()
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(ticc['hcFrac'].dropna(), bins=30, color='green', alpha=0.7)
plt.xlabel('hcFrac Values')
plt.ylabel('Frequency')
plt.title('Histogram of hcFrac')
plt.grid()
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(ticc['ppsRefClock'], ticc['rcFracMa6'], label='rcFracMa6', marker='o')
plt.plot(ticc['ppsRefClock'], ticc['rcFracMa60'], label='rcFracMa60', marker='o')
plt.plot(ticc['ppsRefClock'], ticc['rcFracMa600'], label='rcFracMa600', marker='o')
plt.plot(ticc['ppsRefClock'], ticc['rcFracMa6000'], label='rcFracMa6000', marker='o')
plt.xlabel('ppsRefClock')
plt.ylabel('Rolling Fractional Values')
plt.title('Rolling Fractional Values vs ppsRefClock')
plt.legend()
plt.grid()
plt.show()


In [ ]:
ticc['devRcFracMa6000'] = ticc.rcFrac - ticc.rcFracMa6000
ticc['devRcFracMa600' ] = ticc.rcFrac - ticc.rcFracMa600
ticc['devRcFracMa60'  ] = ticc.rcFrac - ticc.rcFracMa60
ticc['devRcFracMa6'   ] = ticc.rcFrac - ticc.rcFracMa6

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(ticc['ppsRefClock'], ticc['devRcFracMa600'], label='devRcFracMa600', marker='o')
#plt.plot(ticc['ppsRefClock'], ticc['rcFracMa600'], label='rcFracMa600', marker='o')
plt.xlabel('ppsRefClock')
plt.ylabel('Values')
plt.title('devRcFracMa600 and rcFracMa600 vs ppsRefClock')
plt.legend()
plt.grid()
plt.show()


In [ ]:
timTp = pd.read_csv(f"{dirName}/{baseName}.timTpA.csv")
timTp['fn'] = baseName

In [ ]:
timTp

In [ ]:
assert (timTp['timeBase'   ] ==  1).all(), "Not all rows in timTp.timeBase are equal to 1"
assert (timTp['utc'        ] ==  1).all(), "Not all rows in timTp.utc are equal to 1"
assert (timTp['raim'       ] ==  2).all(), "Not all rows in timTp.raim are equal to 2"
assert (timTp['qErrInvalid'] ==  0).all(), "Not all rows in timTp.qErrInvalid are equal to 0"
assert (timTp['TpNotLocked'] ==  0).all(), "Not all rows in timTp.TpNotLocked are equal to 0"
assert (timTp['timeRefGnss'] == 15).all(), "Not all rows in timTp.timeRefGnss are equal to 15"
assert (timTp['utcStandard'] ==  3).all(), "Not all rows in timTp.utcStandard are equal to  3"


In [ ]:
# Constants
gps_epoch = pd.Timestamp("1980-01-06 00:00:00", tz="UTC")
leap_seconds = pd.Timedelta(seconds=18)  # current GPS-UTC offset (2025)

# Vectorized conversion
timTp["epochSec"] = (
    gps_epoch
    + pd.to_timedelta(timTp.week  * 7, unit="D" )
    + pd.to_timedelta(timTp.towMS    , unit="ms")
#    + pd.Timedelta(seconds=0) # Finagle's variable constant to make qErr line up with errors
#    - leap_seconds
)

In [ ]:
timTp

In [ ]:
timTp.dtypes

In [ ]:
tp = pd.merge(ticc, timTp, on="epochSec", how="inner")[:rowLimit]

In [ ]:
tp

In [ ]:
tp['qErrFrac'] = tp.qErr/1e12 # Convert qErr from picoseconds to seconds
tp['rcFracCorr'] = tp.rcFrac+tp.qErrFrac # Correct rcFrac with qErrFrac

In [ ]:
def tdev1sWindow(x: pd.Series) -> float:
    """
    Compute TDEV(1 s) over a 6-sample window.

    Parameters:
        x: A pandas Series of 6 consecutive time error samples (1 s spacing).
    Returns:
        TDEV(1 s) as a float.
    """
    arr = x.to_numpy()
    second_diffs = arr[2:] - 2 * arr[1:-1] + arr[:-2]
    return np.sqrt(np.sum(second_diffs**2) / (6 * (len(arr) - 2)))

# Apply rolling TDEV(1 s)
tp["tdev1s"] = tp["rcFrac"].rolling(window=6).apply(tdev1sWindow, raw=False)
tp["tdevCorr1s"] = tp["rcFracCorr"].rolling(window=6).apply(tdev1sWindow, raw=False)

In [ ]:
plt.figure(figsize=(18, 6))
# Numbers below for baseline3
beg=4000 # Chaos. TDEV(1s) between 2.2 and 2.8
end=4200
#beg=250 # First bridge
#end=650
#beg=425 # First bridge zoom
#end=486
#beg=1800 # Nice hanging bridge
#end=1950
#beg=4500
#end=5500
#plt.plot(tp['ppsHostClock'][beg:end], tp['rcFracMstd6'][beg:end], label='rcFracMstd6', marker='o')
plt.plot(tp['ppsHostClock'][beg:end], tp['tdev1s'][beg:end]*1e9, label='tdev1s', marker='x')
plt.plot(tp['ppsHostClock'][beg:end], tp['tdevCorr1s'][beg:end]*1e9, label='tdev1s', marker='+')
plt.xlabel('ppsHostClock')
plt.ylabel('TDEV(1 s) [ns]')
plt.title('rcFracMstd6 and tdev1s vs ppsHostClock')

ax2 = plt.gca().twinx()
ax2.plot(tp['ppsHostClock'][beg:end], tp['rcFrac'][beg:end], label='rcFrac', color='red', marker='.')
#ax2.legend()

plt.legend()
plt.grid()
plt.show()


In [ ]:
np.savetxt('run1.snap1.corrTie.txt', tp.rcFracCorr, fmt='%.12f')

In [ ]:
tp[['devRcFracMa600', 'qErrFrac']]

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(tp['epochSec'], tp['qErrFrac'], label='qErrFrac', marker='o')
plt.plot(tp['epochSec'], tp['devRcFracMa6'], label='devRcFracMa6', marker='o')
plt.xlabel('epochSec')
plt.ylabel('Values')
plt.title('devRcFracMa600 and qErrFrac vs epochSec')
plt.legend()
plt.grid()
plt.show()


In [ ]:
start = 2100
stop  = 2200
plt.figure(figsize=(18, 6))
plt.plot(tp['epochSec'][start:stop], tp['qErrFrac'][start:stop], label='qErrFrac', marker='o')
plt.plot(tp['epochSec'][start:stop], tp['devRcFracMa60'][start:stop], label='devRcFracMa60', marker='o')
plt.xlabel('epochSec')
plt.ylabel('Values')
plt.title('devRcFracMa600 and qErrFrac vs epochSec (Selected Rows)')
plt.legend()
plt.grid()
plt.show()


In [ ]:
# Compute TDEV and MTIE on uncorrected and corrected PPS
tauTdUc  , tdevUc  , tdevErrUc  , nTdUc   = allantools.tdev(tp.rcFrac    , rate=1.0, taus='all')
tauMtUc  , mtieUc  , mtieErrUc  , nMtUc   = allantools.mtie(tp.rcFrac    , rate=1.0)
tauTdCorr, tdevCorr, tdevErrCorr, nTdCorr = allantools.tdev(tp.rcFracCorr, rate=1.0, taus='all')
tauMtCorr, mtieCorr, mtieErrCorr, nMtCorr = allantools.mtie(tp.rcFracCorr, rate=1.0)
# Convert to nanoseconds
tdevUcNs   = tdevUc   * 1e9
mtieUcNs   = mtieUc   * 1e9
tdevCorrNs = tdevCorr * 1e9
mtieCorrNs = mtieCorr * 1e9

# Compute ADEV on corrected and uncorrected PPS
tauAdUc  , adevUc  , adevErrUc  , nAdUc   = allantools.adev(tp.rcFrac    , rate=1.0, taus='all')
tauAdCorr, adevCorr, adevErrCorr, nAdCorr = allantools.adev(tp.rcFracCorr, rate=1.0, taus='all')

In [ ]:
from matplotlib.ticker import FuncFormatter
sns.set(style="whitegrid")
plt.figure(figsize=(16, 9))
plt.loglog(tauTdUc  , tdevUcNs  , label="Uncorrected TDEV")
plt.loglog(tauTdCorr, tdevCorrNs, label="Corrected TDEV"  )
plt.annotate(f"Uncorrected TDEV(1 s): {tdevUcNs[0]:.3f} ns", xy=(1, tdevUcNs[0]), xytext=(1.1, 3),
             arrowprops=dict(arrowstyle="fancy", ec="black", fc="yellow", lw=0.5))
plt.annotate(f"Corrected TDEV(1 s): {tdevCorrNs[0]:.3f} ns", xy=(1, tdevCorrNs[0]), xytext=(1.1, 0.25),
             arrowprops=dict(arrowstyle="fancy", ec="black", fc="yellow", lw=0.5))
plt.title(r"Impact of u-blox F9T $\mathtt{UBX-TIM-TP\ qErr}$ Corrections on TDEV")
plt.xlabel(r"$\tau$")  # r-string with LaTeX math mode
plt.ylabel(r"$\sigma_x(\tau)$")
plt.grid(which="major", linestyle="-", linewidth=0.5, color="gray")
plt.grid(which="minor", linestyle="--", linewidth=0.5, color="lightgray")
plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{x:g} s"))
plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda y, _: f"{y:g} ns"))

# Shade noise floor
ymin, ymax = plt.ylim() # Save limits before shading
yShadeMin = ymin
yShadeMax = 0.1
plt.axhspan(yShadeMin, yShadeMax, color="lightgray", alpha=0.3)

# Add label centered inside shaded region, remembering that geometric mean is midpoint on log scales
y_center = np.sqrt(yShadeMin * yShadeMax)  # geometric mean for log scale
x_center = np.sqrt(plt.xlim()[0] * plt.xlim()[1])  # vertical center in log scale
plt.text(
    x_center, y_center, "RMS Jitter Floor of TAPR TICC",
    ha="center", va="center",
    fontsize=10, color="black",
    bbox=dict(facecolor="white", alpha=0.7, edgecolor="none")
)
plt.ylim(ymin, ymax) # Restore limits after shading

# ✅ Add text box in the lower-right corner
plt.text(
    0.95, 0.1,                # X & Y position in axes fraction (0–1)
    "Instrument: TAPR TICC\nReference: Geppetto GPSDO with OH300 5ppb OCXO",
    ha="right", va="bottom",   # Align to lower-right corner
    multialignment="left",
    transform=plt.gca().transAxes,  # ✅ Use axes fraction (not data coords)
    fontsize=10,
    bbox=dict(
        facecolor="white",     # Background color
        edgecolor="black",     # Border color
        boxstyle="round,pad=0.3"  # Rounded box with padding
    )
)

plt.legend()
plt.show()

In [ ]:
tp.rcFrac 

In [ ]:
from matplotlib.ticker import FuncFormatter
sns.set(style="whitegrid")
plt.figure(figsize=(16, 9))
plt.loglog(tauMtUc  , mtieUcNs  , label="Uncorrected MTIE")
plt.loglog(tauMtCorr, mtieCorrNs, label="Corrected MTIE"  )
plt.annotate(f"Uncorrected MTIE(1 s): {mtieUcNs[0]:.2f} ns", xy=(1, mtieUcNs[0]), xytext=(1.1, 10.5),
             arrowprops=dict(arrowstyle="fancy", ec="black", fc="yellow", lw=0.5))
plt.annotate(f"Corrected MTIE(1 s): {mtieCorrNs[0]:.2f} ns", xy=(1, mtieCorrNs[0]), xytext=(1.1, 2),
             arrowprops=dict(arrowstyle="fancy", ec="black", fc="yellow", lw=0.5))
plt.title(r"Impact of u-blox F9T $\mathtt{UBX-TIM-TP\ qErr}$ Corrections on MTIE")
plt.xlabel(r"$\tau$")  # r-string with LaTeX math mode
plt.ylabel(r"MTIE")
plt.grid(which="major", linestyle="-", linewidth=0.5, color="gray")
plt.grid(which="minor", linestyle="--", linewidth=0.5, color="lightgray")
plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{x:g} s"))
plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda y, _: f"{y:g} ns"))

# ✅ Add text box in the lower-right corner
plt.text(
    0.95, 0.1,                # X & Y position in axes fraction (0–1)
    "Instrument: TAPR TICC\nReference: Geppetto GPSDO with OH300 5ppb OCXO",
    ha="right", va="bottom",   # Align to lower-right corner
    multialignment="left",
    transform=plt.gca().transAxes,  # ✅ Use axes fraction (not data coords)
    fontsize=10,
    bbox=dict(
        facecolor="white",     # Background color
        edgecolor="black",     # Border color
        boxstyle="round,pad=0.3"  # Rounded box with padding
    )
)

plt.legend()
plt.show()

In [ ]:
from matplotlib.ticker import FuncFormatter
sns.set(style="whitegrid")
plt.figure(figsize=(16, 9))
plt.loglog(tauAdUc  , adevUc  , label="Uncorrected ADEV")
plt.loglog(tauAdCorr, adevCorr, label="Corrected ADEV"  )
plt.annotate(f"Uncorrected ADEV(1 s): {adevUc[0]:.2f} ns", xy=(1, adevUc[0]), xytext=(1.1, 10.5),
             arrowprops=dict(arrowstyle="fancy", ec="black", fc="yellow", lw=0.5))
plt.annotate(f"Corrected ADEV(1 s): {adevCorr[0]:.2f} ns", xy=(1, adevCorr[0]), xytext=(1.1, 2),
             arrowprops=dict(arrowstyle="fancy", ec="black", fc="yellow", lw=0.5))
plt.title(r"Impact of u-blox F9T $\mathtt{UBX-TIM-TP\ qErr}$ Corrections on ADEV XXX fix units and labels")
plt.xlabel(r"$\tau$")  # r-string with LaTeX math mode
plt.ylabel(r"ADEV")
plt.grid(which="major", linestyle="-", linewidth=0.5, color="gray")
plt.grid(which="minor", linestyle="--", linewidth=0.5, color="lightgray")
plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{x:g} s"))
plt.gca().yaxis.set_major_formatter(FuncFormatter(lambda y, _: f"{y:g} ns"))

# ✅ Add text box in the lower-right corner
plt.text(
    0.95, 0.1,                # X & Y position in axes fraction (0–1)
    "Instrument: TAPR TICC\nReference: Geppetto GPSDO with OH300 5ppb OCXO",
    ha="right", va="bottom",   # Align to lower-right corner
    multialignment="left",
    transform=plt.gca().transAxes,  # ✅ Use axes fraction (not data coords)
    fontsize=10,
    bbox=dict(
        facecolor="white",     # Background color
        edgecolor="black",     # Border color
        boxstyle="round,pad=0.3"  # Rounded box with padding
    )
)

plt.legend()
plt.show()